# Next steps in this code

- Change the output file format.  This could go straight into an xarray/netcdf rather than a pickle
- Change the download of timeseries/ data to only load an update to the time series, and then merge with  data that was already downloaded.  Should be faster
- Add Wetlabs data to be downloaded: Not currently downloading.
 

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import json
from io import StringIO
import requests
from setdir import *
import ast # To handle the string conversion when loading json file

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get a token

https://api.c2.noc.ac.uk/charon/tokens/issue


In [17]:
# ISSUE TO RESOLVE: This cannot be in the script, but instead needs to be a 
# text file (myToken.txt) that is ignored in .gitignore and loaded here
#
# Need to copy and paste the token you generated by logging in at the website
# above.
with open("myToken.txt", "r") as myfile:
    myToken = myfile.read().replace('\n', '')
    
from requests.structures import CaseInsensitiveDict
headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = f'Bearer {myToken}'

In [18]:
headers["Authorization"]

'Bearer d038b3578f5642228570ab8dbe3badf6'

## Choose the request options

For time series data, the website is: https://api.c2.noc.ac.uk/timeseries/doc
Check under 'meta/variables' to see what variables can be selected.

Be sure to choose platform: slocum and serial number: unit_409 or unit_398

For positioning data, it's: https://api.c2.noc.ac.uk/positions/doc

An example to download time series data is 
https://api.c2.noc.ac.uk/timeseries/observations/csv_combined?variable=sci_water_pressure&variable=sci_water_temp&variable=sci_water_cond&variable=sci_oxy4_oxygen&variable=sci_bb2flsv9_chl_scaled&variable=derived_salinity&variable=derived_potential_density&variable=derived_potential_temperature&platform_type=slocum&platform_serial=unit_409&reverse_order=false&skip_nulls=false&cached=false


In [19]:
api_root = 'https://api.c2.noc.ac.uk/'

# Choose the unit number (unit_409 or unit_398 for TERIFIC2)
platform = 'slocum'
unit_list = ['unit_398', 'unit_409']

# Choose a start date.  Earliest valid data is 2021-12-12, but there were some in-air measurements before
dstart = '2021-12-12T00%3A00%3A00'
dstart = '2021-12-25T00%3A00%3A00'


# Choose a bunch of variable names
var_list = ['sci_water_pressure', 'sci_water_temp', 'sci_water_cond',
            'sci_oxy4_oxygen','sci_bb2flsv9_chl_scaled', 'derived_salinity',
            'derived_potential_density', 'derived_potential_temperature']
# ISSUE: No wetlabs data
# Hmm. removed the wetlabs because it seemed to remove some of the 
# data/discretise it so that not all the T&S came down
var_list = ['sci_water_pressure', 'sci_water_temp', 'sci_water_cond',
            'sci_oxy4_oxygen','derived_salinity', 'derived_potential_density',
            'derived_potential_temperature', 'm_gps_lon', 'm_gps_lat']

var_str = ''
for i in var_list:
    var_str = var_str+'variable='+i+'&'

    
print(var_str)

# Choose which of the api websites to use.. Not sure how many there are but for these data, it's timeseries/
api_choice = 'timeseries/observations/'

# Using csv_combined_transposed rather than csv_combined since it seems to 
# help with getting all the data (not just when the wetlabs was on)
format_choice = 'csv_combined_transposed?'



variable=sci_water_pressure&variable=sci_water_temp&variable=sci_water_cond&variable=sci_oxy4_oxygen&variable=derived_salinity&variable=derived_potential_density&variable=derived_potential_temperature&variable=m_gps_lon&variable=m_gps_lat&


## Loop through both vehicles

Goes through the serial numbers in unit_list and loads the data.  Not sure whether I want to cut the start, but will decide later.
Saves a pickle with the data in ../data/interim/

In [21]:
# ISSUE: Set up a boolean to decide whether to use a time limited download (from dstart to present) or the full dataset including in-air measurements

# Check that the output directory exists
# If not, then exit
outpath = cat_interim_path('')
if not os.path.isdir(outpath):
   sys.exit(1)


for i in unit_list:
    #--------------------------------------------------------------
    # Format the request
    #--------------------------------------------------------------
    # Time limited
    opt_str = f'from={dstart}&{var_str}platform_type={platform}&platform_serial={i}&reverse_order=false&skip_nulls=false&cached=false'
    start_yyyymmdd = '_'+str.replace(dstart[0:10],'-','')
    
    # No time limiting - Download everything
    opt_str = f'{var_str}platform_type={platform}&platform_serial={i}&reverse_order=false&skip_nulls=false&cached=false'
    start_yyyymmdd = ''

    # Concatinate request string
    url = api_root+api_choice+format_choice+opt_str

    #--------------------------------------------------------------
    # Request the data - save as text in variable 'resp'
    #--------------------------------------------------------------
    resp = requests.get(url, headers=headers)

    # Print the response code (200 is good.  If you get something else, token 
    # may need refreshing)
    print(resp.status_code)

    # Parse the 'resp' string into a dataFrame
    aa = resp.content.decode("utf-8") 
    data_df = pd.read_csv(StringIO(aa)) # Get rid of the leading b
    # Sort by time
    data_df = data_df.sort_values(['timestamp'])

    # Print a little table to the screen
    print(data_df.head(3))

    #--------------------------------------------------------------
    # Format the output file name (and path in ../01-data/01-raw/
    #--------------------------------------------------------------

    # Get a string for today (could be date range in data instead)
    #yyyymmdd = str(today.year)+str(today.month)+str(today.day)
    yyyymmdd = datetime.date.today().strftime('%Y%m%d')

    outfile = i+start_yyyymmdd+'_'+yyyymmdd+'.pkl'
    outfile_with_path = cat_raw_path(outfile)
    data_df.to_pickle(outfile_with_path)

    


200
            timestamp  sci_water_pressure  sci_water_temp  sci_water_cond  \
104578  1511974680000                 NaN             NaN             NaN   
104579  1511974740000                 NaN             NaN             NaN   
104580  1511974800000            0.013333        3.113483        2.926518   

        sci_oxy4_oxygen  derived_salinity  derived_potential_density  \
104578       353.584991               NaN                        NaN   
104579              NaN               NaN                        NaN   
104580       454.519501               NaN                        NaN   

        derived_potential_temperature  m_gps_lon  m_gps_lat  
104578                            NaN        NaN        NaN  
104579                            NaN -40.124973 -52.690977  
104580                            NaN        NaN        NaN  
200
       timestamp  sci_water_pressure  sci_water_temp  sci_water_cond  \
0  1513347840000                 NaN             NaN             NaN   
1 

In [22]:
# Decide format for netcdf output
data_df

,timestamp,sci_water_pressure,sci_water_temp,sci_water_cond,sci_oxy4_oxygen,derived_salinity,derived_potential_density,derived_potential_temperature,m_gps_lon,m_gps_lat
0,1513347840000,NaN,NaN,NaN,NaN,-0.001908,NaN,NaN,NaN,NaN
1,1513347900000,NaN,NaN,NaN,NaN,-0.001908,NaN,NaN,-1.395115,50.892547
2,1513347960000,0.011000,18.818850,4.542040,275.781494,33.908916,1024.233187,18.818830,-1.395115,50.892547
3,1513348020000,0.016333,18.817966,4.542038,275.478841,33.909608,1024.233940,18.817938,NaN,NaN
4,1513348080000,0.188500,18.814750,4.541878,275.437169,33.910291,1024.235349,18.814419,-1.395285,50.893083
...,...,...,...,...,...,...,...,...,...,...
68899,1642478580000,1.254500,3.174600,3.141305,389.697495,34.502377,1027.471770,3.173829,NaN,NaN
68900,1642478640000,0.565000,3.182150,3.141900,389.312500,34.505394,1027.473431,3.181803,NaN,NaN
68901,1642478760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-57.100867,59.574315
68902,1642478880000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-57.100732,59.574358


# Get the position data separately

Exmaple of a request
https://api.c2.noc.ac.uk/positions/positions?platform_type=slocum&platform_serial=unit_409&source_type=internal&time_order=descending




In [24]:
# Hmm. removed the wetlabs because it seemed to remove some of the data/discretise it so that not all the T&S came down
api_root = 'https://api.c2.noc.ac.uk/'

   
# Choose which of the api websites to use.. Not sure how many there are but for these data, it's timeseries/
api_choice = 'positions/'
format_choice = 'positions?'


# Choose the unit number (unit_409 or unit_398 for TERIFIC2)
platform = 'slocum'
unit_list = ['unit_398', 'unit_409']

# Choose a start date.  Earliest valid data is 2021-12-12, but there were some in-air measurements before
dstart = '2021-12-12T00%3A00%3A00'
dstart = '2021-12-25T00%3A00%3A00'


for i in unit_list:
    # Time limited
    opt_str = f'from={dstart}&platform_type={platform}&platform_serial={i}&source_type=internal&time_order=descending'
    start_yyyymmdd = '_'+str.replace(dstart[0:10],'-','')
    
    # No time limiting
    opt_str = f'platform_type={platform}&platform_serial={i}&source_type=internal&time_order=descending'
    start_yyyymmdd = ''

    # Concatinate request string
    url = api_root+api_choice+format_choice+opt_str

    # Get the data
    resp = requests.get(url, headers=headers)

    # Print the response code (200 is good.  If you get something else, token may need refreshing)
    print(resp.status_code)

    # Parse the string into a dataFrame
    json_string = resp.content.decode("utf-8") # Get rid of the leading b
    bb = ast.literal_eval(json_string)
    bb = bb[0]
    glider_path = pd.DataFrame(bb['positions']['internal'])
    glider_path.head()

    # Format the output file name (and path in ../data/interim/)

    # Get a string for today (could be date range in data instead)
#    today = datetime.datetime.now()
#    yyyymmdd = str(today.year)+str(today.month)+str(today.day)
    yyyymmdd = datetime.datetime.now().strftime('%Y%m%d')

    outfile = i+'_positions'+start_yyyymmdd+'_'+yyyymmdd+'.pkl'
    outfile_with_path = cat_raw_path(outfile)
    glider_path.to_pickle(outfile_with_path)

    



200
200
